In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd 

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [17]:
df_train = pd.read_csv("train_set.csv")
df_train
df_train["label"] = df_train["label"].apply(lambda x: 1 if x == "sadness" else 0)   # sadness = 1, happy = 0

In [19]:
y_train = df_train["label"]
X_train = df_train["id_img"]

In [21]:
df_test = pd.read_csv("test_set.csv")
X_test = df_test["id_img"]

In [22]:
y_train = np.asarray(y_train)

AttributeError: 'int' object has no attribute 'lower'

In [29]:
import pandas as pd 

batch_size = 32
img_height = 224
img_width = 224

data_dir = "train"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6176 files belonging to 2 classes.
Using 4941 files for training.
Found 6176 files belonging to 2 classes.
Using 1235 files for validation.


In [30]:
x_test.shape

(4117, 224, 224)

In [31]:
from tensorflow.keras.applications.vgg16 import VGG16

batch_size = 32
img_height = 224
img_width = 224

base_model = VGG16(input_shape = (img_height, img_width, 3),
                  include_top=False,    # false porque ahora la vamos a entrenar nosotros. No nos cogemos todos los pesos
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False 

    
##### FULLY CONNECTED LAYER #####
# creamos una pequeña red al final del modelo para entrenarlo para nuestros datos

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x) # esta x se refiere a la de antes

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Aquí creamos un modelo concatenado, el ya existente y nuestras nuevas capas
model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [32]:
vgghist = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = 10,
                    verbose=1)

Epoch 1/10
155/155 [==============================] - ETA: 0s - loss: 3.2726 - acc: 0.7486

In [9]:
trainPredict

array([[0.46030033],
       [0.03381228],
       [0.01557949],
       ...,
       [0.9999906 ],
       [0.8213645 ],
       [0.00156283]], dtype=float32)

In [16]:


tokenizer = Tokenizer(num_words=3000)
# for human-friendly printing
labels = ['negative', 'positive']

In [ ]:
# y luego el de test
testPredict = model.predict(test_ds)


In [ ]:
# concatenamos antes
#predicted = np.concatenate((trainPredict,testPredict),axis=0)

In [11]:
# sample
sample = pd.read_csv("sample_submission.csv")
sample

,id_img,label
0,18341,sadness
1,13176,sadness
2,23945,happy
3,15968,happy
4,18382,happy
...,...,...
4112,8966,sadness
4113,12111,sadness
4114,16629,sadness
4115,24322,sadness
